In [1]:
import sys
sys.path.append('./..')
sys.path.append('./img2img')

import argparse
import os
# import setGPU
from src.datasets import get_dataset, DATASETS, get_num_classes, get_corrupted_dataset, load_parallel_model
from src.img2img.img2img_models import img2img_get_model, get_size
from src.core import Smooth, TSmooth
from time import time
# from src.grad2n import gradnorm_batch, gradnorm_batch_reduced
from collections import OrderedDict
from src.img2img.corruption import Corruption
# from src.analyze import plot_certified_accuracy, Line, ApproximateAccuracy
import torch
import datetime
from src.architectures import get_architecture
import numpy as np
from tqdm import tqdm

In [2]:
# !pip install -U scikit-image==0.18.0

In [3]:
sys.path.append('./../..')
from architectures import get_architecture

In [4]:
# smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

In [5]:
%env CUDA_VISIBLE_DEVICES=6
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_VISIBLE_DEVICES=6
env: CUDA_LAUNCH_BLOCKING=1


In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
parser = argparse.ArgumentParser(description='Certify many examples')
parser.add_argument("--dataset",
                    default='cifar10',choices=DATASETS, help="which dataset")
parser.add_argument("--base_classifier", type=str,

                    default=None,
                    help="path to saved pytorch model of base classifier")
# parser.add_argument("sigma", type=float, help="noise hyperparameter")
parser.add_argument("--outfile", type=str,
                    default='./logs/certify_results',
                    help="output file")


parser.add_argument("--batch", type=int, default=512, help="batch size")
parser.add_argument("--skip", type=int, default=30
                    , help="how many examples to skip")
parser.add_argument("--max", type=int, default=-1, help="stop after this many examples")
parser.add_argument("--split", choices=["train", "test"], default="test", help="train or test set")
parser.add_argument("--N0", type=int, default=100)
parser.add_argument("--N", type=int, default=1000, help="number of samples to use")
parser.add_argument("--alpha", type=float, default=1e-3, help="failure probability")





parser.add_argument('--corrupt',type=str,default=['none','gaussian_blur','motion_blur','zoom_blur','rotate','translate','contrast','pixelate','jpeg',][1],
                    help=' The corruption type for training')
parser.add_argument('--add_noise',type=float, default=0.0)
parser.add_argument('--noise_dst',default=["none","gaussian","exp","uniform","folded_gaussian"][2],type=str)
parser.add_argument('--noise_sd', default=0.8, type=float,
                    help="standard deviation of Gaussian noise for data augmentation")


parser.add_argument('--partial_min',default=0.0, type=float,
                    help = "Minimal of certify range")
parser.add_argument('--partial_max',default=1.0, type=float,
                    help = "Maximal of certify range")



parser.add_argument('--arch', type=str,
                    default=["edsr","unet","runet"][2])

args = parser.parse_args([])

In [8]:
from torch.utils.data import DataLoader, Dataset

def make_our_dataset_v2(data, idxes):
    images = []
    labels = []
    k = 0
    for i in tqdm(range(len(idxes))):
        image, label = data[idxes[i]]
        images.append(image.numpy())
        
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    dataset = CustomAudioDataset(images, labels)
    return dataset

class CustomAudioDataset(Dataset):

    def __init__(self, images, labels):
        self.images = torch.tensor(images)
        self.labels = torch.tensor(labels)
        
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label =self.labels[idx]
        return image, label

In [9]:
torch.randn([1]).to("cuda")

tensor([-0.3356], device='cuda:0')

In [10]:
np.random.seed(42)
NUM_IMAGES_FOR_TEST = 500
cifar10_ds = get_dataset('cifar10', 'test')
idxes2 = np.random.choice(len(cifar10_ds), NUM_IMAGES_FOR_TEST , replace=False)

our_cifar10_500 = make_our_dataset_v2(cifar10_ds, idxes2)
dataloader_cifar10 = DataLoader(our_cifar10_500, shuffle=False, batch_size=1, num_workers=6, pin_memory=False)

Files already downloaded and verified


100%|████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 6897.01it/s]


In [11]:
np.random.seed(42)

cifar100_ds = get_dataset('cifar100', 'test')
idxes3 = np.random.choice(len(cifar100_ds), NUM_IMAGES_FOR_TEST , replace=False)

our_cifar100_500 = make_our_dataset_v2(cifar100_ds, idxes3)
dataloader_cifar100 = DataLoader(our_cifar100_500, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)

Files already downloaded and verified


100%|████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 6745.10it/s]


In [12]:
from datasets_utils import get_normalize_layer

In [13]:
from archs.cifar_resnet import resnet as resnet_cifar

def get_architecture(arch: str, dataset: str, device) -> torch.nn.Module:
    """ Return a neural network (with random weights)

    :param arch: the architecture - should be in the ARCHITECTURES list above
    :param dataset: the dataset - should be in the datasets.DATASETS list
    :return: a Pytorch module
    """
    if arch == "resnet50" and dataset == "imagenet":
#         model = torch.nn.DataParallel(resnet50(pretrained=False)).cuda()
#         cudnn.benchmark = True
        model = resnet50(pretrained=False).to(device)
    elif arch == "cifar_resnet20":
        model = resnet_cifar(depth=20, num_classes=10).to(device)
    elif arch == "cifar_resnet110":
        model = resnet_cifar(depth=110, num_classes=10).to(device)
    elif arch == "cifar100_resnet110":
        model = resnet_cifar(depth=110, num_classes=100).to(device)
    elif arch == "fashion_22full":
        model = Conv2FC2full()
        model = model.to(device)
    elif arch == "fashion_22simple":
        model = Conv2FC2simple().to(device)
    elif arch == "mnist_43":
        model = Conv4FC3().to(device)
    normalize_layer = get_normalize_layer(dataset, device=device)
    return torch.nn.Sequential(normalize_layer, model)


# Gaussian Blur with exponentially distributed parameters

In [14]:
args.corrupt = 'gaussian_blur'
args.noise_sd = 0.2
args.noise_dst = "exp"
args.partial_max = 4

## CIFAR-10

In [ ]:
# args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.3_0.3/consistency/checkpoint.pth.tar"
# args.base_classifier = "../../new_results/cifar100_resnet110_cifar100_cb_cifar100_cb_checkpoint.pth.tar"
# args.base_classifier = '../../tss_weights/cifar10/resnet110/translation/noise_15.0/consistency/checkpoint.pth.tar'
args.base_classifier = '../../tss_weights/cifar10/resnet110/expgaussian/lamb5/consistency/checkpoint.pth.tar'
args.dataset = 'cifar10'
dataset = our_cifar10_500



device = torch.device('cuda')
checkpoint = torch.load(args.base_classifier, map_location=device)
base_classifier = get_architecture(checkpoint["arch"], args.dataset, device=device)
base_classifier.load_state_dict(checkpoint["state_dict"])


img_size, param_size = get_size(args)

model_t = img2img_get_model(args, img_size, param_size,parallel=False)

trans_path = None
if trans_path is not None:
    state_dict = torch.load(trans_path)['state_dict']
    model_t.load_state_dict(state_dict)
# model_t = load_parallel_model(model_t, state_dict)

# to device
base_classifier.to(device)
base_classifier.eval()
model_t.to(device)
model_t.eval()


#load dataset

# corruptor
corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

# create the smooothed classifier g
smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

# prepare output file
f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

tot, tot_good, tot_correct = 0, 0, 0

# for gaussian smooth
attack_radius = args.partial_max


# dataset = get_dataset(args.dataset, args.split)
dataset = our_cifar10_500
for i in tqdm(range(len(dataset))):

#     # only certify every args.skip examples, and stop after args.max examples
#     if i % args.skip != 0:
#         continue
#     if i == args.max:
#         break

    (x, label) = dataset[i]

    before_time = time()
    x = x.cuda()
    prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


    correct = (prediction == label).item()
    cond1 = radius * args.noise_sd > args.partial_max
#     print(cond1.dtype)
#     print(correct.dtype)
    good = (radius * args.noise_sd > args.partial_max)&correct

    tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
    after_time = time()
    time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
    print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

f.close()

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()

idx	label	predict	radius	good	correct	time


  0%|▏                                                                                           | 1/500 [00:05<49:36,  5.96s/it]

0	2	2	3.52577	False	True	0:00:05.964549


  0%|▎                                                                                           | 2/500 [00:11<49:23,  5.95s/it]

1	1	1	4.28542	False	True	0:00:05.939858


...

26	2	2	3.64991	False	True	0:00:06.016910


  6%|█████                                                                                      | 28/500 [02:46<46:31,  5.91s/it]

27	4	4	4.28542	False	True	0:00:05.778266


  6%|█████▎                                                                                     | 29/500 [02:52<46:07,  5.88s/it]

28	5	5	4.28542	False	True	0:00:05.779467


In [ ]:
cifar_10_res = tot_good/tot

In [19]:
print(cifar_10_res)

0.0


## CIFAR-100

In [16]:
# args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.3_0.3/consistency/checkpoint.pth.tar"
# args.base_classifier = "../../new_results/cifar100_resnet110_cifar100_cb_cifar100_cb_checkpoint.pth.tar"
# args.base_classifier = '../../new_results/trcifar100_resnet110_cifar100_tr_cifar100_trans_checkpoint.pth.tar'
args.base_classifier = '../../new_results/cifar100_resnet110_cifar100_blur_exp_cifar100_blur_exp_checkpoint.pth.tar'
args.corrupt = 'gaussian_blur'
args.dataset = 'cifar100'
args.noise_sd = 0.2
dataset = our_cifar100_500



device = torch.device('cuda')


model = get_architecture(arch="cifar100_resnet110", dataset="cifar100", device=device)
checkpoint = torch.load(args.base_classifier, map_location=device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model = model.to(device)

base_classifier = model


img_size, param_size = get_size(args)

model_t = img2img_get_model(args, img_size, param_size,parallel=False)

trans_path = None
if trans_path is not None:
    state_dict = torch.load(trans_path)['state_dict']
    model_t.load_state_dict(state_dict)
# model_t = load_parallel_model(model_t, state_dict)

# to device
base_classifier.to(device)
base_classifier.eval()
model_t.to(device)
model_t.eval()


#load dataset

# corruptor
corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

# create the smooothed classifier g
smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

# prepare output file
f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

tot, tot_good, tot_correct = 0, 0, 0

# for gaussian smooth
attack_radius = args.partial_max




for i in tqdm(range(len(dataset))):

#     # only certify every args.skip examples, and stop after args.max examples
#     if i % args.skip != 0:
#         continue
#     if i == args.max:
#         break

    (x, label) = dataset[i]

    before_time = time()
    x = x.cuda()
    prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


    correct = (prediction == label).item()
    cond1 = radius * args.noise_sd > args.partial_max
#     print(cond1.dtype)
#     print(correct.dtype)
    good = (radius * args.noise_sd > args.partial_max)&correct

    tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
    after_time = time()
    time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
    print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

f.close()

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()

idx	label	predict	radius	good	correct	time


  0%|▎                                                                                                                                        | 1/500 [00:04<34:54,  4.20s/it]

0	10	-1	0.00000	False	False	0:00:04.196079


  0%|▌                                                                                                                                        | 2/500 [00:08<35:25,  4.27s/it]

1	46	83	0.31154	False	False	0:00:04.314959


...

497	67	67	0.93992	False	True	0:00:06.277587


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 499/500 [45:32<00:05,  5.58s/it]

498	74	-1	0.00000	False	False	0:00:05.863283


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [45:37<00:00,  5.48s/it]

499	29	-1	0.00000	False	False	0:00:04.930530
Total 500 Certified 0 Certified Acc 0.0 Test Acc 0.346


In [ ]:
2

In [20]:
## Gsmooth

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

Total 500 Certified 0 Certified Acc 0.0 Test Acc 0.756


In [15]:


f = open(args.outfile+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()